In [ ]:
!pip install --upgrade langchain openai -q   #install required libraries
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is 

In [ ]:
import os
import openai
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from llama_index import VectorStoreIndex, SimpleDirectoryReader


In [ ]:
directory = 'path for pdf document'

In [ ]:
documents= SimpleDirectoryReader(directory).load_data()

In [ ]:
os.environ["OPENAI_API_KEY"] = "your API Key"

In [ ]:
from IPython.display import HTML, display #Code to wrap output text in screen window

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader


documents = SimpleDirectoryReader(directory).load_data() #load the document


index = VectorStoreIndex.from_documents(documents) #create the index


query_engine = index.as_query_engine() #create the query engine


response = query_engine.query("What is this text about?")  #A sample query asking about the document
print(response)


This text is about a boy who is on a journey to follow his Personal Legend. He encounters a stranger who tests his courage and speaks about the importance of courage in understanding the Language of the World. The stranger also warns the boy about the challenges and dangers of the desert. The boy is reminded of the old king by what the stranger says. Additionally, the text mentions the boy's experience of traveling and making new friends, as well as his anticipation of being with a merchant's daughter in three days.


In [ ]:
from llama_index import LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI

# Create a predictor using a custom model
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo"))  #using LLM gpt-3.5 turbo

# Create a service context with the custom predictor
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# Create an index using the service context
custom_llm_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

custom_llm_query_engine = custom_llm_index.as_query_engine()
response = custom_llm_query_engine.query("what is the summary of the book")
print(response)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


LLMPredictor is deprecated, please use LLM instead.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The book follows the journey of a boy who trades his book for a thicker one and enjoys the new wine he bought. He is traveling with his flock of sheep and makes new friends along the way. The boy reads the new book and becomes immersed in the story. He encounters an old man who tries to strike up a conversation, but the boy is more interested in his reading. Later, the boy meets an Englishman who is disappointed by the boy's lack of interest in his research and symbols. The boy continues his journey with a caravan and learns about the war between tribes. The animals are exhausted, and the silence of the night becomes unsettling. The book explores the themes of personal legends and the pursuit of knowledge.


In [ ]:
#Advanced Implementation
from llama_index import get_response_synthesizer
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters
from llama_index.query_engine.retriever_query_engine import (
    RetrieverQueryEngine,
)

# build retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,  #retrieving top 3 similar chunks
    vector_store_query_mode="default",
    alpha=None,
    doc_ids=None,
)

# build query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever, response_synthesizer=get_response_synthesizer()
)

# query
response = query_engine.query("what did the author do growing up?")
print(response.metadata)
print(response)

{'bec63236-51c1-4434-9c4f-db638e826430': {'page_label': '25', 'file_name': 'THE-ALCHEMIST-pdf-free-download.pdf', 'file_path': '/content/gdrive/My Drive/emotional-health/THE-ALCHEMIST-pdf-free-download.pdf', 'file_type': 'application/pdf', 'file_size': 1102910, 'creation_date': '2024-02-09', 'last_modified_date': '2023-08-12', 'last_accessed_date': '2024-01-09'}, '6ce99786-e8d0-4802-afb7-2cb41db5f96e': {'page_label': '19', 'file_name': 'THE-ALCHEMIST-pdf-free-download.pdf', 'file_path': '/content/gdrive/My Drive/emotional-health/THE-ALCHEMIST-pdf-free-download.pdf', 'file_type': 'application/pdf', 'file_size': 1102910, 'creation_date': '2024-02-09', 'last_modified_date': '2023-08-12', 'last_accessed_date': '2024-01-09'}, 'dc41d2dc-87c8-49ec-b2aa-50a65cd183f3': {'page_label': '17', 'file_name': 'THE-ALCHEMIST-pdf-free-download.pdf', 'file_path': '/content/gdrive/My Drive/emotional-health/THE-ALCHEMIST-pdf-free-download.pdf', 'file_type': 'application/pdf', 'file_size': 1102910, 'creatio